In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Import Libraries**

In [ ]:
import numpy as np
import os
import cv2
import pickle
import matplotlib.pyplot as plt
import tensorflow as tf
import random
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# **Data PreProcessing**

In [ ]:
labels = ['bad', 'good']
img_size = 100
def get_training_data(data_dir):
    data = [] 
    for label in labels: 
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img),cv2.IMREAD_GRAYSCALE)
                resized_arr = cv2.resize(img_arr, (img_size, img_size))
                data.append([resized_arr, class_num])
            except Exception as e:
                print(e)
    return np.array(data)

In [ ]:
train = get_training_data('../input/cubesat/training_dataset_v3')
test = get_training_data('../input/cubesat/test_dataset_v5')

# **Data Distribution**

In [ ]:
bad = 0 
good = 0 

for i, j in train:
    if j == 0:
        bad+=1
    else:
        good+=1
        
print('Bad:', bad)
print('good:', good)
print('Bad - Good:', bad-good)


In [ ]:
X = []
y = []

for feature, label in train:
    X.append(feature)
    y.append(label)

for feature, label in test:
    X.append(feature)
    y.append(label)
    


# resize data for deep learning 
X = np.array(X).reshape(-1, img_size, img_size, 1)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.20, random_state=32)

# **Data Reshaping and Normalization**

In [ ]:
X_train = X_train.reshape(X_train.shape[0],-1).T
X_test = X_test.reshape(X_test.shape[0],-1).T
X_train = X_train / 255
X_test = X_test / 255
X_val = X_val / 255
y_train = np.array(y_train).reshape(-1,1)
y_train = y_train.T
y_test = np.array(y_test).reshape(-1,1)
y_test = y_test.T

# **Activation Functions**

# **Sigmoid**
<img src = "https://upload.wikimedia.org/wikipedia/commons/thumb/5/53/Sigmoid-function-2.svg/1280px-Sigmoid-function-2.svg.png" width="400">

In [ ]:
def sigmoid(Z):
    
    A = 1/(1+np.exp(-Z))
    cache = Z
    
    return A, cache

# **Relu**
<img src = "https://sebastianraschka.com/images/faq/relu-derivative/relu_3.png" width="400">

In [ ]:
def relu(Z):
    
    A = np.maximum(0,Z)
    
    assert(A.shape == Z.shape)
    
    cache = Z 
    return A, cache

# **Backward Activation Function**

In [ ]:
def relu_backward(dA, cache):

    
    Z = cache
    dZ = np.array(dA, copy=True)
    
    dZ[Z <= 0] = 0
    
    assert (dZ.shape == Z.shape)
    
    return dZ

In [ ]:
def sigmoid_backward(dA, cache):

    
    Z = cache
    
    s = 1/(1+np.exp(-Z))
    dZ = dA * s * (1-s)
    
    assert (dZ.shape == Z.shape)
    
    return dZ

## **Defining the Layer of The Neural Network**
<img src = "https://i.imgur.com/UX6M1zX.png" width = 800>

In [ ]:
### CONSTANTS DEFINING THE MODEL ####
n_x = 10000     # num_px * num_px * 1
n_h = 7
n_y = 1
layers_dims = (n_x, n_h, n_y)

# **Implementation of Deep Learning Neural Network Model**
<img src = "https://i.imgur.com/cizhNhg.jpeg" width="700">

# **Initialize Weights and Bias**

In [ ]:
def initialize_parameters(layer_dims):

    
    np.random.seed(3)
    parameters = {}
    L = len(layer_dims)            # number of layers in the network

    for l in range(1, L):
        
        parameters['W' + str(l)] = np.random.randn(layer_dims[l],layer_dims[l-1])*0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l],1))
        
        
        assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l-1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))

        
    return parameters

# **Fordward Propagation**

In [ ]:
def linear_forward(A, W, b):

    Z = np.dot(W,A)+b
    
    
    assert(Z.shape == (W.shape[0], A.shape[1]))
    cache = (A, W, b)
    
    return Z, cache

In [ ]:
def linear_activation_forward(A_prev, W, b, activation):

    if activation == "sigmoid":
        # Inputs: "A_prev, W, b". Outputs: "A, activation_cache".
        
        Z, linear_cache = linear_forward(A_prev,W,b)
        A, activation_cache = sigmoid(Z)
        
    
    elif activation == "relu":
        # Inputs: "A_prev, W, b". Outputs: "A, activation_cache".
        
        Z, linear_cache = linear_forward(A_prev,W,b)
        A, activation_cache = relu(Z)
        
    
    assert (A.shape == (W.shape[0], A_prev.shape[1]))
    cache = (linear_cache, activation_cache)

    return A, cache

# **Cost Calculation**

<img src = "https://www.researchgate.net/profile/Kipp-Johnson/publication/331439931/figure/fig2/AS:733179287728129@1551814972455/mpact-of-deep-learning-design-on-learning-effect-of-learning-rate-A-Efficient.png" width = "500">

In [ ]:
def compute_cost(AL, Y):
    
    m = Y.shape[1]

    # Compute loss from aL and y.
    
    cost = -1/m * (np.sum(Y*np.log(AL)+((1-Y)*np.log(1-AL))))
    
    
    cost = np.squeeze(cost)     
    assert(cost.shape == ())
    
    return cost

# **Back Propagation**

In [ ]:
def linear_backward(dZ, cache):

    A_prev, W, b = cache
    m = A_prev.shape[1]

    
    dW = 1/m *(np.dot(dZ,A_prev.T))
    db = 1/m * np.sum(dZ,axis = 1,keepdims = True)
    dA_prev = np.dot(W.T,dZ)
    
    
    
    assert (dA_prev.shape == A_prev.shape)
    assert (dW.shape == W.shape)
    assert (db.shape == b.shape)
    
    return dA_prev, dW, db

In [ ]:
def linear_activation_backward(dA, cache, activation):

    linear_cache, activation_cache = cache
    
    if activation == "relu":
        
        dZ = relu_backward(dA, activation_cache)
        dA_prev, dW, db = (linear_backward(dZ,linear_cache))
        
        
    elif activation == "sigmoid":
        
        dZ = sigmoid_backward(dA, activation_cache)
        dA_prev, dW, db = (linear_backward(dZ,linear_cache))
        
    
    return dA_prev, dW, db

# **Updating the values of Weights and Bias**

In [ ]:
def update_parameters(parameters, grads, learning_rate):

    
    L = len(parameters) // 2 # number of layers in the neural network

    # Update rule for each parameter. Use a for loop.
    
    for l in range(L):
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)]- learning_rate*grads['dW'+str(l+1)]
        parameters["b" + str(l+1)] =parameters["b" + str(l+1)] - learning_rate*grads['db'+str(l+1)]
    
    return parameters

**A model For L Layer Deep Neural Network**

In [ ]:
def L_model_forward(X, parameters):

    caches = []
    A = X
    L = len(parameters) // 2                  # number of layers in the neural network
    
    # Implement [LINEAR -> RELU]*(L-1). Add "cache" to the "caches" list.
    for l in range(1, L):
        A_prev = A 
        A, cache = linear_activation_forward(A_prev, parameters['W' + str(l)], parameters['b' + str(l)], activation = "relu")
        caches.append(cache)
    
    # Implement LINEAR -> SIGMOID. Add "cache" to the "caches" list.
    AL, cache = linear_activation_forward(A, parameters['W' + str(L)], parameters['b' + str(L)], activation = "sigmoid")
    caches.append(cache)
    
    assert(AL.shape == (1,X.shape[1]))
            
    return AL, caches
    

# **Predict Function**

In [ ]:
def predict(X, y, parameters):

    
    m = X.shape[1]
    n = len(parameters) // 2 # number of layers in the neural network
    p = np.zeros((1, m),dtype=int)
    
    # Forward propagation
    probas, caches = L_model_forward(X, parameters)


    # convert probas to 0/1 predictions
    for i in range(0, probas.shape[1]):
        if probas[0,i] > 0.5:
            p[0,i] = 1
        else:
            p[0,i] = 0

    #print results
    #print ("predictions: " + str(p))
    #print ("true labels: " + str(y))
    acc = np.sum(p == y)/float(m)
        
    return acc

# **Finalizing The Model**

In [ ]:
def two_layer_model(X, Y, layers_dims, learning_rate = 0.0075, num_iterations = 2500, print_cost=False):

    np.random.seed(1)
    grads = {}
    costs = []                              # to keep track of the cost
    m = X.shape[1]                           # number of examples
    (n_x, n_h, n_y) = layers_dims
    
    # Initialize parameters dictionary, by calling one of the functions you'd previously implemented
    
    parameters = initialize_parameters(layers_dims)
    
    
    # Get W1, b1, W2 and b2 from the dictionary parameters.
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    
    # Loop (gradient descent)

    for i in range(0, num_iterations):

        # Forward propagation: LINEAR -> RELU -> LINEAR -> SIGMOID. Inputs: "X, W1, b1, W2, b2". Output: "A1, cache1, A2, cache2".
        
        #minibatches = random_mini_batches(X_train, Y_train, minibatch_size, seed)
        A1, cache1 = linear_activation_forward(X, W1, b1, 'relu')
        A2, cache2 = linear_activation_forward(A1,W2,b2,'sigmoid')
        
        
        # Compute cost

        cost = compute_cost(A2, Y)
        
        
        # Initializing backward propagation
        dA2 = - (np.divide(Y, A2) - np.divide(1 - Y, 1 - A2))
        
        # Backward propagation. Inputs: "dA2, cache2, cache1". Outputs: "dA1, dW2, db2; also dA0 (not used), dW1, db1".

        
        dA1, dW2, db2 =  linear_activation_backward(dA2, cache2, activation='sigmoid')
        dA0, dW1, db1 =  linear_activation_backward(dA1, cache1, activation='relu')
        

        
        # Set grads['dWl'] to dW1, grads['db1'] to db1, grads['dW2'] to dW2, grads['db2'] to db2
        grads['dW1'] = dW1
        grads['db1'] = db1
        grads['dW2'] = dW2
        grads['db2'] = db2
        
        # Update parameters.

        parameters = update_parameters(parameters, grads, learning_rate)


        # Retrieve W1, b1, W2, b2 from parameters
        W1 = parameters["W1"]
        b1 = parameters["b1"]
        W2 = parameters["W2"]
        b2 = parameters["b2"]
        
        # Print the cost every 100 training example
        if print_cost and i % 100 == 0:
            print("Cost after iteration {}: {}".format(i, np.squeeze(cost)))
        if print_cost and i % 100 == 0:
            costs.append(cost)
       
    # plot the cost

    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per hundreds)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()
    
    return parameters

# **Training Model**

In [ ]:
parameters = two_layer_model(X_train, y_train, layers_dims = (n_x, n_h, n_y), num_iterations = 2500, print_cost=True)
predictions_train = print("train accuracy: {} %".format(predict(X_train, y_train, parameters) * 100))
predictions_train = print("test accuracy: {} %".format(predict(X_test, y_test, parameters) * 100))

# **Testing Model**

In [ ]:
test_image1 = cv2.imread('../input/cubesat/test_dataset_v5/good/0.jpg')
test_image = cv2.imread('../input/cubesat/test_dataset_v5/good/14.jpg',cv2.IMREAD_GRAYSCALE)
resized_arr1 = cv2.resize(test_image, (100, 100))
resized_arr1 = np.array(resized_arr1).reshape(-1, img_size, img_size, 1)
resized_arr1 = resized_arr1.reshape(resized_arr1.shape[0],-1).T
resized_arr1 = resized_arr1/255
my_predicted_image = predict(resized_arr1, my_label_y, parameters)

plt.imshow(test_image1)
print ("ACCURACY IS= " + str(np.squeeze(my_predicted_image)*100 )+"%" )
if my_predicted_image>0.5:
    print("There is Good Image")
else:
    print("The Image is Bad Image")

